<a href="https://colab.research.google.com/github/Raghuram-Veeramallu/Astro_Stat_Project2/blob/development/Group3_Project2_DataSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AST/STAT 5731 Project 2
### Research Synopsis
**Group 3**   
Daniel Warshofsky  
Hari Veeramallu  
Jacynda Alatoma  
Nicholas Kruegler

#### Research Question

*Synopsis here*


### Preamble

### Data   

Raw data is available at [Google Drive](https://drive.google.com/file/d/1v6LSAKvkuEjahtOWDNq3riBMLkD7rZD0/view)

##### 1. Loading the data

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# download the dataset to colab / local environment

## for google colab
!gdown 1v6LSAKvkuEjahtOWDNq3riBMLkD7rZD0

## for local environment
# !gdown https://drive.google.com/uc?id=1v6LSAKvkuEjahtOWDNq3riBMLkD7rZD0

Downloading...
From: https://drive.google.com/uc?id=1v6LSAKvkuEjahtOWDNq3riBMLkD7rZD0
To: /content/snaeT1.tsv
100% 47.4k/47.4k [00:00<00:00, 64.4MB/s]


In [6]:
## Load the data

# NOTE: change the data path to the download path mentioned in the cell above
# This is typically your downloads folder
data = pd.read_csv('/content/snaeT1.tsv', sep='\t')

##### 2. Looking at the data

In [7]:
data.head(5)

,zcmb,zhel,mb,e_mb,x1,e_x1,c,e_c,logMst
0,0.503,0.5043,23.002,0.088,1.273,0.150,-0.012,0.030,9.517
1,0.581,0.5820,23.574,0.090,0.974,0.274,-0.025,0.037,9.169
2,0.495,0.4960,22.960,0.088,-0.729,0.102,-0.100,0.030,11.580
3,0.346,0.3470,22.398,0.087,-1.155,0.113,-0.041,0.027,10.821
4,0.678,0.6790,24.078,0.098,0.619,0.404,-0.039,0.067,8.647


**Column Description:**.

| Column  | Description                 |
| ------- | --------------------------- |
| zcmb    | CMB Frame Redshift          |
| zhel    | Heliocentric Redshift       |
| mb      | B band peak magnitude (in mB)|
| e_mb    | Error in mb                 |
| x1      | SALT2 shape (stretch) parameter |
| e_x1    | Error in x1                 |
| c       | SALT2 color parameter       |
| e_c     | Error in c                  |
| logMst  | $Log_{10}$ Host Stellar Mass|

In [9]:
data.describe()

,zcmb,zhel,mb,e_mb,x1,e_x1,c,e_c,logMst
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000
mean,0.323870,0.324449,20.904961,0.115899,0.036589,0.311096,-0.023878,0.038897,9.795342
std,0.276893,0.276748,2.655139,0.018767,0.988554,0.211399,0.084118,0.015694,1.395068
min,0.010000,0.009400,14.148000,0.085000,-2.863000,0.018000,-0.250000,0.012000,5.000000
25%,0.124000,0.124425,19.735500,0.102000,-0.650250,0.148500,-0.084250,0.026000,9.316750
50%,0.229000,0.230500,21.099000,0.114000,0.165500,0.268000,-0.030500,0.035000,10.193500
75%,0.498000,0.499000,23.043000,0.124000,0.775750,0.420000,0.031000,0.050000,10.710250
max,1.299000,1.300000,26.047000,0.175000,2.337000,1.641000,0.260000,0.107000,11.817000


##### 3. Converting the shape and color paramters to distance

Since we want to estimate the relationship between the Distance and the Redshift (CMB), we need to compute the distance from the parameters that we have available to us.

Distance can be computed through Distance Modulii $\mu$, defined as
$$ \mu = m_{B}^{*} - (M_{B} - \alpha X_{1} + \beta C)$$


where,  
$m_{B}^{*}$ is the observed peak magnitude in rest frame B band.  
$M_{B}$ is the absolute magnitude of the supernova (adjusted based on host galaxy's properties). It depends on the host galalxy's stellar mass ($M_{stellar}$).   
$$M_{B} = \begin{cases} M_{B}^{1} & \text{if $M_{stellar} < 10^{10} M_{\odot}$} \\
M_{B}^{1} + \Delta_{m} & \text{otherwise}
\end{cases}$$   
here, $M_{\odot}$ is the mass of the Sun.   
$\alpha, \beta$ are the nuisance parameters that account for the shape of the light curve ($X_{1}$) and the color of the supernova ($C$) respectively.  
$\beta, M_{B}$ are found to be dependent on the host galaxy properties.  


Modulus can be converted into distance using $ d_{L} = 10^{\mu/5 + 1} $.  
($\mu = 5 log_{10}{(d_{L} /10 pc)}$ according to the literature). **TODO: add link here**

Since there are errors associated with each measurement, we need to estimate the distance errors as well.   

Error parameter $\sigma_{\mu}$ = $\sqrt{\sigma_{m_{b}}^{2} + (\alpha \sigma_{X_{1}})^2 + (\beta \sigma_{C})^2}$.  
where $\sigma_{m_{b}}, \sigma_{X_{1}}, \sigma_{C}$ refer to the error terms of $m_{b}, X_{1}$ and $C$ (`e_mb`, `e_x1` and `e_c` from the dataset).  

Error in distance measurements
$$\sigma_{d} = \frac{\sigma_{\mu} . log(10) . |d|}{5}$$

From the literature the values used are, $\alpha = 0.141, \beta = 3.101, M_{B}^{1} = -19.05$ and $\Delta_{M} = -0.07$ (according to the C11 analysis)

The absolute magnitude of type Ia in B-band is consistent, typically around $-19.3 \pm 0.3 $ in magnitude.

In [11]:
# defining the nuiscance parameters
alpha = 0.141
beta = 3.101
MB1 = -19.05
DeltaM = -0.07

In [12]:
# MB
data['MB'] = MB1 + (DeltaM * (data['logMst'] > 10))

# mu
data['dist_moduli'] = data['mb'] - (data['MB'] - alpha * data['x1'] + beta * data['c'])

# distance
data['d'] = 10 ** (data['dist_moduli']/5 + 1)

In [14]:
# distance modulii (mu) error
data['e_mu'] = np.sqrt(data['e_mb'] ** 2 + (alpha * data['e_x1']) ** 2 + (beta * data['e_c']))

# distance error
data['e_d']= (data['e_mu'] * np.log(10) * np.abs(data['d']))/5

In [15]:
data.head(5)

,zcmb,zhel,mb,e_mb,x1,e_x1,c,e_c,logMst,MB,dist_moduli,d,e_mu,e_d
0,0.503,0.5043,23.002,0.088,1.273,0.150,-0.012,0.030,9.517,-19.05,42.268705,2.842765e+09,0.318153,4.165074e+08
1,0.581,0.5820,23.574,0.090,0.974,0.274,-0.025,0.037,9.169,-19.05,42.838859,3.696339e+09,0.352604,6.002121e+08
2,0.495,0.4960,22.960,0.088,-0.729,0.102,-0.100,0.030,11.580,-19.12,42.287311,2.867228e+09,0.317775,4.195922e+08
3,0.346,0.3470,22.398,0.087,-1.155,0.113,-0.041,0.027,10.821,-19.12,41.482286,1.979052e+09,0.302572,2.757603e+08
4,0.678,0.6790,24.078,0.098,0.619,0.404,-0.039,0.067,8.647,-19.05,43.336218,4.647759e+09,0.469698,1.005328e+09


In [19]:
# selecting only the required parameters from here
df = data[['zcmb', 'd', 'e_d']]
df.describe()

,zcmb,d,e_d
count,740.000000,7.400000e+02,7.400000e+02
mean,0.323870,1.881246e+09,3.514203e+08
std,0.276893,1.872763e+09,3.972315e+08
min,0.010000,4.120357e+07,6.331576e+06
25%,0.124000,5.923373e+08,8.447847e+07
50%,0.229000,1.150148e+09,1.957524e+08
75%,0.498000,2.828445e+09,4.563507e+08
max,1.299000,9.490109e+09,2.500113e+09
